To do:
1. double check logstic convergence issues
2. need to re-run rca before running this since top mods are wrong 

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import log_loss, make_scorer
from sklearn.linear_model import RidgeCV, LogisticRegressionCV
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from tqdm.notebook import tqdm
import itertools
import json
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Loading Data

In [2]:
rca = pd.read_csv('../../data/final/rca.csv')
rca

,embed,norm,train_n,p,r2,check
0,SVD_sim_rel,Freq_HAL,4506,300,0.089362,pass
1,SVD_sim_rel,Freq_KF,3776,300,0.036674,pass
2,SVD_sim_rel,Freq_SUBTLEXUS,4450,300,0.156546,pass
3,SVD_sim_rel,Freq_SUBTLEXUK,4472,300,0.060241,pass
4,SVD_sim_rel,Freq_Blog,4652,300,0.098733,pass
...,...,...,...,...,...,...
7301,SGSoftMaxDecoder_SWOW,goals_vanarsdall,959,300,0.403416,pass
7302,SGSoftMaxDecoder_SWOW,movement_vanarsdall,959,300,0.345155,pass
7303,SGSoftMaxDecoder_SWOW,concreteness_vanarsdall,959,300,0.132158,pass
7304,SGSoftMaxDecoder_SWOW,familiarity_vanarsdall,959,300,0.081765,pass


In [3]:
embed_means = rca.groupby('embed').mean(numeric_only=True)

# Adding embed types
with open('../../data/final/dtype_to_embed.json', 'r') as f:
    type_2_embed = json.load(f)
embed_2_type = lambda name: 'text' if name in type_2_embed['text'] else 'behavior' if name in type_2_embed['behavior'] else 'brain'
embed_means['type'] = embed_means.index.map(embed_2_type)

top_n = 2

# ensembling top text
top_text_names = embed_means.query('type == "text"').sort_values('r2', ascending=False).head(top_n).index.tolist()
text_text_names = list(itertools.combinations(top_text_names, r=2))
text_text_names

[('fastText_CommonCrawl', 'GloVe_CommonCrawl')]

In [4]:
# Ensembling top behavior
top_behavior_names = embed_means.query('type == "behavior"').sort_values('r2', ascending=False).head(top_n).index.tolist()
text_behavior_names =  []
for text_name in top_text_names:
    for behavior_name in top_behavior_names:
        text_behavior_names.append((text_name, behavior_name))
text_behavior_names

[('fastText_CommonCrawl', 'PPMI_SVD_SWOW'),
 ('fastText_CommonCrawl', 'norms_sensorimotor'),
 ('GloVe_CommonCrawl', 'PPMI_SVD_SWOW'),
 ('GloVe_CommonCrawl', 'norms_sensorimotor')]

In [5]:
standarize = lambda df: (df - df.mean()) / df.std()

# Loading embeddings
embeds = {}
for name in top_text_names + top_behavior_names:
    embeds[name] = pd.read_pickle(f'../../data/processed/pulled_embeds/{name}.pkl')

{name: embed.shape for name, embed in embeds.items()}

{'fastText_CommonCrawl': (88953, 300),
 'GloVe_CommonCrawl': (88408, 300),
 'PPMI_SVD_SWOW': (11781, 300),
 'norms_sensorimotor': (36851, 11)}

In [6]:
meta = pd.read_csv('../../data/final/norm_metadata.csv', index_col=0)
meta['associated_embed'] = meta['associated_embed'].str.split(' ')

norms = pd.read_csv('../../data/final/norms.csv', index_col=0)
norms

/var/folders/st/5gbrfvhn687dxwjl5_xg21t40000gq/T/ipykernel_50490/1726302198.py:4: DtypeWarning: Columns (22,23,170) have mixed types. Specify dtype option on import or set low_memory=False.
  norms = pd.read_csv('../../data/final/norms.csv', index_col=0)


,Freq_HAL,Freq_KF,Freq_SUBTLEXUS,Freq_SUBTLEXUK,Freq_Blog,Freq_Twitter,Freq_News,Freq_CobW,Freq_CobS,CD_SUBTLEXUS,...,iconicity_winter_2017,living_vanarsdall,thought_vanarsdall,reproduction_vanarsdall,person_vanarsdall,goals_vanarsdall,movement_vanarsdall,concreteness_vanarsdall,familiarity_vanarsdall,imageability_vanarsdall
'em,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.3617,1.9138,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'neath,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
're,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.9031,1.6335,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'shun,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'tis,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.4771,0.6021,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
trappy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vocalise,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
listened..to.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
spoke..to.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Cross Validation

In [9]:
standardize = lambda df: (df - df.mean()) / df.std()

def mcfadden_r2_binary(y_true, y_pred_proba):
    # Compute the log-likelihood of the model
    ll_model = -log_loss(y_true, y_pred_proba, normalize=False)

    # Compute the log-likelihood of the null model (predicting the mean)
    probas_null = np.full_like(y_pred_proba, fill_value=y_true.mean(), dtype=np.float64)
    ll_null = -log_loss(y_true, probas_null, normalize=False)

    # Calculate McFadden's R2
    pseudo_r2 = 1 - (ll_model / ll_null)
    return pseudo_r2

def mcfadden_r2_multiclass(y_true, y_pred_proba):
    # Convert y_true to a binary matrix representation (one-hot encoding)
    lb = LabelBinarizer()
    y_true_binary = lb.fit_transform(y_true)

    # Compute the log-likelihood of the model
    ll_model = -log_loss(y_true_binary, y_pred_proba, normalize=False)

    # Compute the log-likelihood of the null model (predicting class proportions)
    class_proportions = y_true_binary.mean(axis=0)
    probas_null = np.array([class_proportions] * len(y_true))
    ll_null = -log_loss(y_true_binary, probas_null, normalize=False)

    # Calculate McFadden's R2
    pseudo_r2 = 1 - (ll_model / ll_null)
    return pseudo_r2

def best_logistic_solver(y, dtype):
    """
    Pick the fastest 'l2'-compatible for LogisticCV the given data based on a few heuristics.
    """
    if len(y) < 1000:  # Arbitrary threshold for "small" datasets
        if dtype == 'binary':
            return 'liblinear'
        else:
            return 'lbfgs'
    else:
        return 'saga' 

def process_categorical(X_1, X_2, y):
    """Removes classes with too few observations"""
    min_class_n = outer_cv * inner_cv
    classes_to_keep = y.value_counts()[y.value_counts() >= min_class_n].index
    to_keep_bool = y.isin(classes_to_keep)
    X_1, X_2, y = X_1.loc[to_keep_bool], X_2.loc[to_keep_bool], y.loc[to_keep_bool]
    return X_1, X_2, y

def check_data(X_names, y_name, y, dtype):
    associated_embeds = meta.loc[y_name, 'associated_embed']
    if isinstance(associated_embeds, list):
        if set(X_names) & set(associated_embeds):  # if either is associated
            return 'associated_embed'
    elif (1 - (1 / outer_cv)) * len(y) < 600:  # ensures n>p 
        return 'too few observations'  
    elif (dtype != 'continuous') and (len(y.unique()) < 2):
        return 'too few classes (of sufficient size)'
    else:
        return 'pass'
    
def top_or_bottom(best, possibles):
    if isinstance(best, float):
        if best == possibles[0]:
            return 'bottom'
        elif best == possibles[-1]:
            return 'top'
        else:
            return 'pass'
    else:
        for b in best:
            result = top_or_bottom(b, possibles)
            if result != 'pass':
                return result
        return 'pass'

def check_penalty(estim, X, y, dtype):
    """Checks that none of the best penalties are at either extreme of alphas or Cs""" 
    stratify = y if dtype != 'continuous' else None
    train_size = 1 - (1 / outer_cv)
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, train_size=train_size, shuffle=True, stratify=stratify
    )
    estim.fit(X_train, y_train)
                      
    if dtype == 'continuous':
        result = top_or_bottom(estim.alpha_, alphas)
    else:
        result = top_or_bottom(estim.C_, Cs)
    
    if result != 'pass':
        print(f'Penalty at {result} of range')
        return result


def cross_val(estim, X, y, dtype):
    if dtype == 'continuous':
        return cross_val_score(estim, X, y, cv=outer_cv, n_jobs=n_jobs, scoring='r2')
    elif dtype == 'binary':
        return cross_val_score(estim, X, y, cv=outer_cv, n_jobs=n_jobs, scoring=binary_scorer)
    else:
        return cross_val_score(estim, X, y, cv=outer_cv, n_jobs=n_jobs, scoring=multiclass_scorer)


# Ridge
min_alpha, max_alpha = -3, 6 
alphas = np.logspace(-3, 6,  max_alpha - min_alpha + 1)
ridge = RidgeCV(alphas=alphas)

# Logistic hyperparameters
Cs = 1 / alphas
inner_cv = 5
penalty = 'l2'

# Scorers
binary_scorer = make_scorer(mcfadden_r2_binary, needs_proba=True, greater_is_better=True)
multiclass_scorer = make_scorer(mcfadden_r2_multiclass, needs_proba=True, greater_is_better=True)

# outer_cv setting 
outer_cv, n_jobs = 5, 8

In [10]:
# RCA
rca = []
for (text_name, behavior_name) in text_behavior_names:
    
    # Loading text-text baseline embedding
    text_text_embed = pd.concat([embeds[name] for name in top_text_names], axis=1, join='inner')
    text_text_embed.columns = list(range(text_text_embed.shape[1]))
    text_text_name = '&'.join(top_text_names)

    # Loading text-behavior embedding
    text_behavior_embed = pd.concat([embeds[text_name], embeds[behavior_name]], axis=1, join='inner')
    text_behavior_embed.columns = list(range(text_behavior_embed.shape[1]))
    text_behavior_name = f'{text_name}&{behavior_name}'
    
    # Aligning embedding to have same vocab for fair comparison
    text_text_embed, text_behavior_embed = text_text_embed.align(text_behavior_embed, axis='index', join='inner')
    
    # Standardizing
    text_text_embed, text_behavior_embed = standardize(text_text_embed), standardize(text_behavior_embed)
       
    for norm_name in tqdm(norms.columns, desc=text_behavior_name):
        
        # Aligning embeddings with norm
        norm = norms[norm_name].dropna()
        tt_embed, norm = text_text_embed.align(norm, axis='index', join='inner')
        tb_embed, norm = text_behavior_embed.align(norm, axis='index', join='inner')
        
        # Checking norm dtype 
        norm_dtype = meta.loc[norm_name, 'type']
        
        # Solvers, scoring, estimators ir categorical or continuous
        if norm_dtype in ['binary', 'multiclass']: # categorical
            tt_embed, tb_embed, norm = process_categorical(tt_embed, tb_embed, norm)
            
            # may have switched form multi to bin after processing
            norm_dtype = 'binary' if len(norm.unique()) == 2 else 'multiclass'
            
            # Cross validation settings for logistic regression
            solver = best_logistic_solver(norm, norm_dtype)
            
            # Defining logistic regression 
            estimator = LogisticRegressionCV(
                Cs=Cs, penalty=penalty, cv=StratifiedKFold(inner_cv), solver=solver
            )
        else: # continuous
            estimator, scoring = ridge, 'r2'
            
        # Cross validation
        embed_names = top_text_names + [behavior_name]
        data_check = check_data(embed_names, norm_name, norm, norm_dtype)
        if data_check == 'pass':
            penalty_check = check_penalty(estimator, tb_embed, norm, norm_dtype)
            text_text_scores = cross_val(estimator, tt_embed, norm, norm_dtype)
            text_behavior_scores = cross_val(estimator, tb_embed, norm, norm_dtype)
        else:
            text_text_scores, text_behavior_scores = [np.nan] * outer_cv, [np.nan] * outer_cv
            penalty_check = [np.nan]*outer_cv
            
        # Saving
        train_n = int(((outer_cv - 1) / outer_cv) * len(norm))
        for text_score, text_behavior_score in zip(text_text_scores, text_behavior_scores):
            rca.append([
                text_text_name, text_behavior_name, norm_name, train_n, text_score, 
                text_behavior_score, data_check, penalty_check
            ])
 
 
rca = pd.DataFrame(
    rca, columns=['text_text_name', 'text_behavior_name', 'norm', 'train_n', 'r2_text',  'r2_ensemb', 'data_check', 'penalty_check']
)
# rca.to_csv('../../data/final/rca_ensemb.csv', index=False)
rca

fastText_CommonCrawl&PPMI_SVD_SWOW:   0%|          | 0/281 [00:00<?, ?it/s]

/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/zhussain/opt/anaconda3/envs/PsychEmbeddings/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


KeyboardInterrupt: 